In [ ]:
pip install rdkit

In [ ]:
pip install chainer-chemistry

In [ ]:
import logging
from rdkit import RDLogger
from chainer_chemistry import datasets

lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

logging.basicConfig(level=logging.INFO)

In [ ]:
dataset_filepath = datasets.get_qm9_filepath()

print('dataset_filepath =', dataset_filepath)

In [ ]:
from chainer_chemistry.dataset.preprocessors.ggnn_preprocessor import \
    GGNNPreprocessor
    
preprocessor = GGNNPreprocessor()
dataset, dataset_smiles = datasets.get_qm9(preprocessor, labels=None, return_smiles=True)

In [ ]:
print('dataset information...')
print('dataset', type(dataset), len(dataset))

print('smiles information...')
print('dataset_smiles', type(dataset_smiles), len(dataset_smiles))

In [ ]:
print('length of dataset:', len(dataset))

In [ ]:
# Print first 5 SMILES strings
for i in range(5):
    print(f"SMILES {i+1}: {dataset_smiles[i]}")


In [ ]:
from rdkit import Chem


for smile in dataset_smiles:
    mol = Chem.MolFromSmiles(smile)
    if mol is not None:  
        print(Chem.MolToInchiKey(mol))  


In [ ]:
print("Total number of SMILES:", len(dataset_smiles))
print("Type of dataset_smiles:", type(dataset_smiles))


In [ ]:
from IPython.display import display

molecules = [Chem.MolFromSmiles(smile) for smile in dataset_smiles[:5] if smile]
img = Draw.MolsToGridImage(molecules, molsPerRow=5, useSVG=True)  
display(img)


In [ ]:
pip install torch-geometric

In [ ]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdmolops
import torch
from torch_geometric.data import Data

def smiles_to_graph(smiles_list):
    data_list = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:  
            continue
        
        # Get atom features
        atomic_nums = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
        aromatics = [atom.GetIsAromatic() for atom in mol.GetAtoms()]
        hybridizations = [atom.GetHybridization().real for atom in mol.GetAtoms()]
        num_hydrogens = [atom.GetTotalNumHs() for atom in mol.GetAtoms()]
        
        # Create node features matrix
        x = np.column_stack([atomic_nums, aromatics, hybridizations, num_hydrogens])
        
        # Get edges and bond types
        edge_index = []
        edge_attr = []
        for bond in mol.GetBonds():
            start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
            edge_index.append([start, end])
            edge_index.append([end, start])  # Since the graph is undirected
            
            # Encode bond type
            bond_type = bond.GetBondType()
            if bond_type == Chem.rdchem.BondType.SINGLE:
                edge_attr.append([1, 0, 0, 0])
                edge_attr.append([1, 0, 0, 0])  # Same for reverse direction
            elif bond_type == Chem.rdchem.BondType.DOUBLE:
                edge_attr.append([0, 1, 0, 0])
                edge_attr.append([0, 1, 0, 0])
            elif bond_type == Chem.rdchem.BondType.TRIPLE:
                edge_attr.append([0, 0, 1, 0])
                edge_attr.append([0, 0, 1, 0])
            elif bond_type == Chem.rdchem.BondType.AROMATIC:
                edge_attr.append([0, 0, 0, 1])
                edge_attr.append([0, 0, 0, 1])

        
        edge_index = torch.tensor(edge_index, dtype=torch.long).t()
        edge_attr = torch.tensor(edge_attr, dtype=torch.float)
        
        
        data = Data(x=torch.tensor(x, dtype=torch.float), edge_index=edge_index, edge_attr=edge_attr)
        data_list.append(data)
        
    return data_list

# # Example usage:
# smiles = ["C1=CC=CC=C1", "CCO"]  # Replace this list with your dataset_smiles
# graph_data = smiles_to_graph(smiles)


In [ ]:
# Assuming 'dataset_smiles' is a list of SMILES strings
print("First few SMILES entries:")
for i, smiles in enumerate(dataset_smiles[:5]): 
    print(f"{i + 1}: {smiles}")


In [ ]:
graph_data = smiles_to_graph(dataset_smiles)


for graph in graph_data[:5]:  
    print(graph)
    print("Number of nodes:", graph.num_nodes)
    print("Number of edges:", graph.num_edges)
    print("Node features:", graph.x)
    print("Edge index:", graph.edge_index)
    print("Edge attribute",graph.edge_attr)

In [ ]:
# def split_data_indices(smiles_list, dataset, train_size=10000, valid_size=1000, test_size=1000):
#     # Generate indices and shuffle them
#     indices = np.arange(len(smiles_list))
#     np.random.shuffle(indices)
    
#     # Split indices for train, validation, and test
#     train_indices = indices[:train_size]
#     valid_indices = indices[train_size:train_size + valid_size]
#     test_indices = indices[train_size + valid_size:train_size + valid_size + test_size]
    
#     # Subset the SMILES strings and the dataset
#     train_smiles = smiles_list[train_indices]
#     valid_smiles = smiles_list[valid_indices]
#     test_smiles = smiles_list[test_indices]

#     train_dataset = dataset[train_indices]
#     valid_dataset = dataset[valid_indices]
#     test_dataset = dataset[test_indices]
    
#     return train_smiles, valid_smiles, test_smiles, train_dataset, valid_dataset, test_dataset

# # Assuming `dataset_smiles` and `dataset` are numpy arrays
# train_smiles, valid_smiles, test_smiles, train_dataset, valid_dataset, test_dataset = split_data_indices(dataset_smiles, dataset)

# # Now, you can process the SMILES to graph data, and use the dataset arrays as needed
# train_graph_data = smiles_to_graph(train_smiles)
# valid_graph_data = smiles_to_graph(valid_smiles)
# test_graph_data = smiles_to_graph(test_smiles)

# # Optionally print the size of each dataset to verify
# print(f"Training graphs: {len(train_graph_data)}, Training dataset: {len(train_dataset)}")
# print(f"Validation graphs: {len(valid_graph_data)}, Validation dataset: {len(valid_dataset)}")
# print(f"Testing graphs: {len(test_graph_data)}, Testing dataset: {len(test_dataset)}")


In [ ]:
# import numpy as np

# def split_data(graph_data, dataset_smiles, train_size=10000, val_size=1000, test_size=1000):
#     assert len(graph_data) >= train_size + val_size + test_size, "Dataset is too small for the desired split sizes."
#     assert len(graph_data) == len(dataset_smiles), "Mismatch in the length of graph data and SMILES data."

#     indices = np.random.permutation(len(dataset_smiles))
    
#     train_indices = indices[:train_size]
#     val_indices = indices[train_size:train_size+val_size]
#     test_indices = indices[train_size+val_size:train_size+val_size+test_size]
    
#     train_data = [graph_data[i] for i in train_indices]
#     val_data = [graph_data[i] for i in val_indices]
#     test_data = [graph_data[i] for i in test_indices]
    
#     train_smiles = dataset_smiles[train_indices]
#     val_smiles = dataset_smiles[val_indices]
#     test_smiles = dataset_smiles[test_indices]
    
#     return (train_data, train_smiles), (val_data, val_smiles), (test_data, test_smiles)

# (train_data, train_smiles), (val_data, val_smiles), (test_data, test_smiles) = split_data(graph_data, dataset_smiles)

# print(f"Training data size: {len(train_data)}")
# print(f"Validation data size: {len(val_data)}")
# print(f"Testing data size: {len(test_data)}")


In [ ]:

import numpy as np
from rdkit import Chem
import torch
from torch_geometric.data import Data

def quantum_encode(atom, use_atomic_num=True, use_nh=True, use_aromaticity=True, use_hybridization=True):
    z = atom.GetAtomicNum() if use_atomic_num else None
    nh = atom.GetTotalNumHs() if use_nh else None
    aromatic = atom.GetIsAromatic() if use_aromaticity else None
    hybridization = atom.GetHybridization() if use_hybridization else None

    # Default angles
    ry_angle, rz_angle = 0, 0

    if use_atomic_num and not(use_nh) and use_aromaticity and use_hybridization:
        ry_angle = (2 * z - 7) * np.pi / 4 if z else 0
        rz_angle = (-1)**(2 * hybridization - 1) * np.pi / 6 if hybridization else 0
    elif use_atomic_num and use_nh:
        ry_angle = (2 * z - 7) * np.pi / 4 if z else 0
        rz_angle = 2 * np.pi * nh / 5 if nh else 0
    elif use_atomic_num:
        if z in [5, 6, 7]:
            ry_angle = np.cos(-1 / 3)
        if z == 6:
            rz_angle = 2 * np.pi / 3
        elif z == 7:
            rz_angle = -2 * np.pi / 3

    return torch.tensor([ry_angle, rz_angle], dtype=torch.float)

def process_molecule(smiles, use_atomic_num=True, use_nh=True, use_aromaticity=True, use_hybridization=True):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    features = [quantum_encode(atom, use_atomic_num, use_nh, use_aromaticity, use_hybridization) for atom in mol.GetAtoms()]
    x = torch.stack(features) if features else torch.empty(0)
    edge_index = torch.tensor(np.array(Chem.rdmolops.GetAdjacencyMatrix(mol).nonzero()), dtype=torch.long)
    
    edge_attrs = []
    for bond in mol.GetBonds():
        start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        # Encode bond type as a vector, e.g., [single, double, triple, aromatic]
        bond_type = [bond.GetBondTypeAsDouble()]
        edge_attrs.append(bond_type)
        # Add reverse bond to ensure undirected graph is properly handled
        edge_attrs.append(bond_type)

    edge_attr = torch.tensor(edge_attrs, dtype=torch.float).reshape(-1, 1) if edge_attrs else torch.empty((0, 1))

    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)


feature_flags = {
    'use_atomic_num': True,
    'use_nh': True,
    'use_aromaticity': True,
    'use_hybridization': True
}

# Process the molecules
encoded_train_data = [process_molecule(smiles, **feature_flags) for smiles in dataset_smiles if process_molecule(smiles, **feature_flags) is not None]

# first few 
for data in encoded_train_data[:5]:
    print(data)
    print("Number of nodes:", data.num_nodes)
    print("Number of edges:", data.num_edges)
    print("Node features:\n", data.x)
    print("Edge index:\n", data.edge_index)
    print("Edge attribute:\n",data.edge_attr)


In [ ]:

print("First few entries of the dataset:")
for i in range(min(5, len(dataset))):  
    print(f"Entry {i}: {dataset[i]}")


In [ ]:
# import torch
# from rdkit import Chem
# from rdkit.Chem import rdmolops
# from torch_geometric.data import Data

# def smiles_to_graph(smiles_list, dataset):
#     data_list = []
#     for smiles, entry in zip(smiles_list, dataset):
#         properties = entry[2] if len(entry) > 2 else None
#         mol = Chem.MolFromSmiles(smiles)
#         if mol is None or properties is None:
#             continue
#         node_features = []
#         for atom in mol.GetAtoms():
#             z = atom.GetAtomicNum()
#             nh = atom.GetTotalNumHs()
#             aromatic = atom.GetIsAromatic()
#             hybridization = atom.GetHybridization().real
#             node_features.append([z, nh, aromatic, hybridization])
           
           
#         x = torch.tensor(node_features, dtype=torch.float)

#         # Edge indices and attributes
#         edge_index = []
#         edge_attr = []
#         for bond in mol.GetBonds():
#             start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
#             edge_index.append([start, end])
#             edge_index.append([end, start])  # Since the graph is undirected

#             # Bond type encoding as one-hot
#             bond_type = [0, 0, 0, 0]  # [single, double, triple, aromatic]
#             if bond.GetBondType() == Chem.rdchem.BondType.SINGLE:
#                 bond_type[0] = 1
#             elif bond.GetBondType() == Chem.rdchem.BondType.DOUBLE:
#                 bond_type[1] = 1
#             elif bond.GetBondType() == Chem.rdchem.BondType.TRIPLE:
#                 bond_type[2] = 1
#             elif bond.GetBondType() == Chem.rdchem.BondType.AROMATIC:
#                 bond_type[3] = 1
#             edge_attr.append(bond_type)
#             edge_attr.append(bond_type)  # Same for reverse direction for undirected graphs

#         # Convert lists to tensors for PyTorch Geometric
#         edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
#         edge_attr = torch.tensor(edge_attr, dtype=torch.float)
        
        
#         homo_lumo_gap = properties[7]  
#         y = torch.tensor([homo_lumo_gap], dtype=torch.float)

        
#         data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
#         data_list.append(data)
        
#     return data_list

# graph_data = smiles_to_graph(dataset_smiles, dataset)

# for graph in graph_data[:5]:
#     print("Graph:", graph)
#     print("Number of nodes:", graph.num_nodes)
#     print("Number of edges:", graph.num_edges)
#     print("Node features:\n", graph.x)
#     print("Edge attributes:\n", graph.edge_attr)
#     print("Target property (HOMO-LUMO gap):", graph.y)


In [ ]:
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp

def quantum_encode(features, use_atomic_num=True, use_nh=False, use_aromaticity=True, use_hybridization=True):
    encoded_params = []
    for feature in features:
        z, nh, aromatic, hybridization = feature.tolist()
        ry_angle, rz_angle = 0, 0

        if use_atomic_num and not use_nh and use_aromaticity and use_hybridization:
            ry_angle = (2 * z - 7) * np.pi / 4 if z is not None else 0
            rz_angle = (-1)**(2 * hybridization - 1) * np.pi / 6 if hybridization is not None else 0
        elif use_atomic_num and use_nh:
            ry_angle = (2 * z - 7) * np.pi / 4 if z is not None else 0
            rz_angle = 2 * np.pi * nh / 5 if nh is not None else 0
        elif use_atomic_num:
            if z in [5, 6, 7]:
                ry_angle = np.cos(-1 / 3)
            if z == 6:
                rz_angle = 2 * np.pi / 3
            elif z == 7:
                rz_angle = -2 * np.pi / 3

        encoded_params.append([ry_angle, rz_angle])
    return np.array(encoded_params)

def RZZ(theta, wires):
    qml.CNOT(wires=wires)
    qml.RZ(theta, wires=wires[1])
    qml.CNOT(wires=wires)

# Define the quantum circuit
def circuit(features, bond_info, use_atomic_num=True, use_nh=False, use_aromaticity=True, use_hybridization=True):
    num_qubits = len(features)  # Define the number of qubits
    dev = qml.device('default.qubit', wires=num_qubits)

    @qml.qnode(dev, interface='numpy')  # Use NumPy interface for compatibility
    def quantum_circuit(encoded_params):
        for i in range(num_qubits):
            qml.RY(encoded_params[i][0], wires=i)
            qml.RZ(encoded_params[i][1], wires=i)
        
        # Entangling unitaries between each pair of qubits
        for i in range(num_qubits - 1):
            if bond_info[i] == 'single':
                RZZ(params['theta_link_single'][i], wires=[i, i+1])
            elif bond_info[i] == 'double':
                RZZ(params['theta_link_double'][i], wires=[i, i+1])

        return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

    encoded_params = quantum_encode(features, use_atomic_num, use_nh, use_aromaticity, use_hybridization)
    return quantum_circuit(encoded_params)

# Example usage
features = np.array([[6, 1, 0, 3], [7, 1, 2, 1]])  # Example features for 2 nodes
bond_info = ['single']  # Bond information between the two nodes

# Define random parameters for RZZ, assuming there's only one bond
params = {
    'theta_link_single': np.random.random(1),
    'theta_link_double': np.random.random(1)
}

output = circuit(features, bond_info)
print("Circuit output:", output)


In [ ]:
import numpy as np
from rdkit import Chem
import torch
from torch_geometric.data import Data
import pennylane as qml

# Define function to convert SMILES to graph data
def smiles_to_graph(smiles_list):
    data_list = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            continue

        atomic_nums = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
        aromatics = [atom.GetIsAromatic() for atom in mol.GetAtoms()]
        hybridizations = [int(atom.GetHybridization()) for atom in mol.GetAtoms()]
        num_hydrogens = [atom.GetTotalNumHs() for atom in mol.GetAtoms()]

        x = np.column_stack([atomic_nums, aromatics, hybridizations, num_hydrogens])
        edge_index = []
        edge_attr = []
        bond_info = []

        for bond in mol.GetBonds():
            start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
            bond_type = bond.GetBondType()
            if bond_type == Chem.rdchem.BondType.SINGLE:
                edge_attr.append([1, 0, 0, 0])
                bond_info.append('single')
            elif bond_type == Chem.rdchem.BondType.DOUBLE:
                edge_attr.append([0, 1, 0, 0])
                bond_info.append('double')
            elif bond_type == Chem.rdchem.BondType.TRIPLE:
                edge_attr.append([0, 0, 1, 0])
                bond_info.append('triple')
            elif bond_type == Chem.rdchem.BondType.AROMATIC:
                edge_attr.append([0, 0, 0, 1])
                bond_info.append('aromatic')

            edge_index.append([start, end])
            edge_index.append([end, start])  # Since the graph is undirected

        edge_index = torch.tensor(edge_index, dtype=torch.long).t()
        edge_attr = torch.tensor(edge_attr, dtype=torch.float)
        x = torch.tensor(x, dtype=torch.float)

        data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, bond_info=bond_info)
        data_list.append(data)

    return data_list

In [ ]:
num_entries_to_print = 5 
for i in range(min(num_entries_to_print, len(dataset))):
        data_tuple = dataset[i]
        print(f"Entry {i+1}:")
        for j, component in enumerate(data_tuple):
            print(f"  Component {j+1}: {component}")

In [ ]:
import numpy as np
import torch
from torch_geometric.data import Data
from rdkit import Chem
import pennylane as qml
import matplotlib.pyplot as plt

def smiles_to_graph_and_targets(smiles_list, dataset):
    data_list = []
    targets = []
    for smiles, data in zip(smiles_list, dataset):
        # Correctly accessing the properties array which is nested within the data
        properties = data[2] if len(data) > 2 else None
        
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            print(f"Failed to parse SMILES: {smiles}")
            continue

        atomic_nums = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
        aromatics = [atom.GetIsAromatic() for atom in mol.GetAtoms()]
        hybridizations = [int(atom.GetHybridization()) for atom in mol.GetAtoms()]
        num_hydrogens = [atom.GetTotalNumHs() for atom in mol.GetAtoms()]

        x = np.column_stack([atomic_nums, aromatics, hybridizations, num_hydrogens])
        edge_index = []
        edge_attr = []
        bond_info = []

        for bond in mol.GetBonds():
            start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
            bond_type = bond.GetBondType()
            bond_label = {Chem.rdchem.BondType.SINGLE: 'single',
                          Chem.rdchem.BondType.DOUBLE: 'double',
                          Chem.rdchem.BondType.TRIPLE: 'triple',
                          Chem.rdchem.BondType.AROMATIC: 'aromatic'}.get(bond_type, 'single')
            bond_info.append(bond_label)
            edge_attr.append([1 if bond_label == bl else 0 for bl in ['single', 'double', 'triple', 'aromatic']])
            edge_index.append([start, end])
            edge_index.append([end, start])

        edge_index = torch.tensor(edge_index, dtype=torch.double).t()
        edge_attr = torch.tensor(edge_attr, dtype=torch.double)
        x = torch.tensor(x, dtype=torch.float)

        data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, bond_info=bond_info)
        data_list.append(data)

        # Properly checking and extracting the target value
        if properties is not None and len(properties) > 7 and properties[7] is not None:
            target_value = properties[7]
            #print(target_value)
            targets.append(torch.tensor([target_value], dtype=torch.double))
        else:
            print(f"No valid target value for SMILES: {smiles} with properties {properties}")

    return data_list, targets


# Load and print the data
graph_data, targets = smiles_to_graph_and_targets(dataset_smiles, dataset)


def quantum_encode(features, use_atomic_num=True, use_nh=False, use_aromaticity=True, use_hybridization=True):
    encoded_params = []
    for feature in features:
        z, nh, aromatic, hybridization = feature.tolist()
        ry_angle, rz_angle = 0, 0

        if use_atomic_num and not use_nh and use_aromaticity and use_hybridization:
            ry_angle = (2 * z - 7) * np.pi / 4 if z is not None else 0
            rz_angle = (-1)**(2 * hybridization - 1) * np.pi / 6 if hybridization is not None else 0
        elif use_atomic_num and use_nh:
            ry_angle = (2 * z - 7) * np.pi / 4 if z is not None else 0
            rz_angle = 2 * np.pi * nh / 5 if nh is not None else 0
        elif use_atomic_num:
            if z in [5, 6, 7]:
                ry_angle = np.cos(-1 / 3)
            if z == 6:
                rz_angle = 2 * np.pi / 3
            elif z == 7:
                rz_angle = -2 * np.pi / 3

        encoded_params.append([ry_angle, rz_angle])
    return np.array(encoded_params)

def RZZ(theta, wires):
    qml.CNOT(wires=wires)
    qml.RZ(theta, wires=wires[1])
    qml.CNOT(wires=wires)

def quantum_circuit(params, features, bond_info):
    num_qubits = len(features)
    dev = qml.device('default.qubit', wires=num_qubits)

    @qml.qnode(dev, interface='torch')
    def circuit(features):
        # Applying initial rotations
        for i in range(num_qubits):
            qml.RY(features[i][0], wires=i)
            qml.RZ(features[i][1], wires=i)

        # Apply parameterized gates based on bond info
        param_indices = {bond: 0 for bond in set(bond_info)}
        for i in range(num_qubits - 1):
            bond_type = bond_info[i]
            if bond_type in params and param_indices[bond_type] < len(params[bond_type]):
                RZZ(params[bond_type][param_indices[bond_type]], wires=[i, i + 1])
                param_indices[bond_type] += 1

        return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

    encoded_features = quantum_encode(features)
    expectation_values = circuit(torch.tensor(encoded_features, dtype=torch.double))
    expectation_values_tensor = torch.tensor(expectation_values, dtype=torch.double)
    mean_expectation = torch.mean(expectation_values_tensor)

    # Calculate the readout using r0, r1, and mean_expectation
    readout = r0 + r1 * mean_expectation / len(features)
    return readout


def allocate_params(bond_info):
    params = {}
    for bond in set(bond_info):
        count = bond_info.count(bond)
        params[bond] = torch.randn(count, dtype=torch.double, requires_grad=True)
        print(f"Allocated {count} parameters for {bond} bonds.")
    return params

def train_and_plot(graph_data, targets):
    r0 = torch.tensor(0.1, dtype=torch.double, requires_grad=True)
    r1 = torch.tensor(0.2, dtype=torch.double, requires_grad=True)
    optimizer = torch.optim.Adam([r0, r1], lr=0.01)
    loss_func = torch.nn.MSELoss()
    losses = []

    for epoch in range(150):
        total_loss = 0
        for data, target in zip(graph_data, targets):
            optimizer.zero_grad()
            params = allocate_params(data.bond_info)
            readout = quantum_circuit(params, data.x, data.bond_info)
            loss = loss_func(readout, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            if epoch == 0:
                print(f"Initial target value: {target.item()}")

        epoch_loss = total_loss / len(graph_data)
        losses.append(epoch_loss)
        print(f"Epoch {epoch}: Loss {epoch_loss}")

    plt.figure(figsize=(10, 5))
    plt.plot(losses, label='Training Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training Loss Curve')
    plt.legend()
    plt.grid(True)
    plt.show()

train_and_plot(graph_data, targets)

In [ ]:
import numpy as np
import torch
from torch_geometric.data import Data
from rdkit import Chem
import pennylane as qml
import matplotlib.pyplot as plt

def smiles_to_graph_and_targets(smiles_list, dataset):
    data_list = []
    targets = []
    for smiles, data in zip(smiles_list, dataset):
        properties = data[2] if len(data) > 2 else None
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            continue

        atomic_nums = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
        aromatics = [atom.GetIsAromatic() for atom in mol.GetAtoms()]
        hybridizations = [int(atom.GetHybridization()) for atom in mol.GetAtoms()]
        num_hydrogens = [atom.GetTotalNumHs() for atom in mol.GetAtoms()]

        x = np.column_stack([atomic_nums, aromatics, hybridizations, num_hydrogens])
        edge_index = []
        edge_attr = []
        bond_info = []

        for bond in mol.GetBonds():
            start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
            bond_type = bond.GetBondType()
            bond_label = {Chem.rdchem.BondType.SINGLE: 'single',
                          Chem.rdchem.BondType.DOUBLE: 'double',
                          Chem.rdchem.BondType.TRIPLE: 'triple',
                          Chem.rdchem.BondType.AROMATIC: 'aromatic'}.get(bond_type, 'single')
            bond_info.append(bond_label)
            edge_attr.append([1 if bond_label == bl else 0 for bl in ['single', 'double', 'triple', 'aromatic']])
            edge_index.append([start, end])
            edge_index.append([end, start])

        edge_index = torch.tensor(edge_index, dtype=torch.double).t()
        edge_attr = torch.tensor(edge_attr, dtype=torch.double)
        x = torch.tensor(x, dtype=torch.double)
        data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, bond_info=bond_info)
        data_list.append(data)

        if properties is not None and len(properties) > 7 and properties[7] is not None:
            target_value = properties[7]
            targets.append(torch.tensor([target_value], dtype=torch.double))

    return data_list, targets

def split_data(graph_data, targets, train_size, val_size, test_size):
    indices = np.random.permutation(len(graph_data))
    train_indices = indices[:train_size]
    val_indices = indices[train_size:train_size + val_size]
    test_indices = indices[train_size + val_size:train_size + val_size + test_size]

    train_data = [graph_data[i] for i in train_indices]
    train_targets = [targets[i] for i in train_indices]
    val_data = [graph_data[i] for i in val_indices]
    val_targets = [targets[i] for i in val_indices]
    test_data = [graph_data[i] for i in test_indices]
    test_targets = [targets[i] for i in test_indices]

    return (train_data, train_targets), (val_data, val_targets), (test_data, test_targets)

def quantum_encode(features, use_atomic_num=True, use_nh=False, use_aromaticity=True, use_hybridization=True):
    encoded_params = []
    for feature in features:
        z, nh, aromatic, hybridization = feature.tolist()
        ry_angle, rz_angle = 0, 0

        if use_atomic_num and not use_nh and use_aromaticity and use_hybridization:
            ry_angle = (2 * z - 7) * np.pi / 4 if z is not None else 0
            rz_angle = (-1)**(2 * hybridization - 1) * np.pi / 6 if hybridization is not None else 0
        elif use_atomic_num and use_nh:
            ry_angle = (2 * z - 7) * np.pi / 4 if z is not None else 0
            rz_angle = 2 * np.pi * nh / 5 if nh is not None else 0
        elif use_atomic_num:
            if z in [5, 6, 7]:
                ry_angle = np.cos(-1 / 3)
            if z == 6:
                rz_angle = 2 * np.pi / 3
            elif z == 7:
                rz_angle = -2 * np.pi / 3

        encoded_params.append([ry_angle, rz_angle])
    return np.array(encoded_params)

def RZZ(theta, wires):
    qml.CNOT(wires=wires)
    qml.RZ(theta, wires=wires[1])
    qml.CNOT(wires=wires)

def quantum_circuit(r0,r1,params, features, bond_info):
    num_qubits = len(features)
    dev = qml.device('default.qubit', wires=num_qubits)

    @qml.qnode(dev, interface='torch')
    def circuit(features):
        for i in range(num_qubits):
            qml.RY(features[i][0], wires=i)
            qml.RZ(features[i][1], wires=i)

        param_indices = {bond: 0 for bond in set(bond_info)}
        for i in range(num_qubits - 1):
            bond_type = bond_info[i]
            if bond_type in params and param_indices[bond_type] < len(params[bond_type]):
                RZZ(params[bond_type][param_indices[bond_type]], wires=[i, i + 1])
                param_indices[bond_type] += 1

        return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

    encoded_features = quantum_encode(features)
    expectation_values = circuit(torch.tensor(encoded_features, dtype=torch.double))
    expectation_values_tensor = torch.tensor(expectation_values, dtype=torch.double)
    mean_expectation = torch.mean(expectation_values_tensor)

    readout = r0 + r1 * mean_expectation / len(features)
    return readout

def allocate_params(bond_info):
    params = {}
    for bond in set(bond_info):
        count = bond_info.count(bond)
        params[bond] = torch.randn(count, dtype=torch.double, requires_grad=True)
        print(f"Allocated {count} parameters for {bond} bonds.")
    return params

def train_and_plot(train_data, train_targets, val_data, val_targets):
    r0 = torch.tensor(0.1, dtype=torch.double, requires_grad=True)
    r1 = torch.tensor(0.2, dtype=torch.double, requires_grad=True)
    optimizer = torch.optim.Adam([r0, r1], lr=0.01)
    loss_func = torch.nn.MSELoss()
    train_losses = []
    val_losses = []

    for epoch in range(150):
        total_train_loss = 0
        for data, target in zip(train_data, train_targets):
            optimizer.zero_grad()
            params = allocate_params(data.bond_info)
            readout = quantum_circuit(r0,r1,params, data.x, data.bond_info)
            loss = loss_func(readout, target)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        total_val_loss = 0
        with torch.no_grad():
            for data, target in zip(val_data, val_targets):
                params = allocate_params(data.bond_info)
                readout = quantum_circuit(r0,r1,params, data.x, data.bond_info)
                loss = loss_func(readout, target)
                total_val_loss += loss.item()

        epoch_train_loss = total_train_loss / len(train_data)
        epoch_val_loss = total_val_loss / len(val_data)
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)
        print(f"Epoch {epoch}: Train Loss {epoch_train_loss}, Validation Loss {epoch_val_loss}")

    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss Curve')
    plt.legend()
    plt.grid(True)
    plt.show()

# Load the dataset
graph_data, targets = smiles_to_graph_and_targets(dataset_smiles, dataset)

# Split the data
(train_data, train_targets), (val_data, val_targets), (test_data, test_targets) = split_data(graph_data, targets, 10000, 1000, 1000)

# Train and plot
train_and_plot(train_data, train_targets, val_data, val_targets)


In [ ]:
import pennylane as qml
from pennylane import numpy as np
import torch

def quantum_encode(features, use_atomic_num=True, use_nh=True, use_aromaticity=True, use_hybridization=True):
    encoded_params = []
    for feature in features:
        z = feature[0] if use_atomic_num else None
        nh = feature[1] if use_nh else None
        aromatic = feature[2] if use_aromaticity else None
        hybridization = feature[3] if use_hybridization else None

        ry_angle, rz_angle = 0, 0

        if use_atomic_num and not use_nh and use_aromaticity and use_hybridization:
            ry_angle = (2 * z - 7) * np.pi / 4 if z is not None else 0
            rz_angle = (-1)**(2 * hybridization - 1) * np.pi / 6 if hybridization is not None else 0
        elif use_atomic_num and use_nh:
            ry_angle = (2 * z - 7) * np.pi / 4 if z is not None else 0
            rz_angle = 2 * np.pi * nh / 5 if nh is not None else 0
        elif use_atomic_num:
            if z in [5, 6, 7]:
                ry_angle = np.cos(-1 / 3)
            if z == 6:
                rz_angle = 2 * np.pi / 3
            elif z == 7:
                rz_angle = -2 * np.pi / 3
        encoded_params.append([ry_angle, rz_angle])

    return torch.tensor(encoded_params, dtype=torch.float)


flags = {
    'use_atomic_num': True,
    'use_nh': True,
    'use_aromaticity': False,
    'use_hybridization': True
}

for i, graph in enumerate(graph_data):
    num_qubits = graph.num_nodes
    dev = qml.device('default.qubit', wires=num_qubits)
    
    @qml.qnode(dev)
    def quantum_encoding_circuit(params):
        for j in range(len(params)):
            qml.RY(params[j, 0], wires=j)
            qml.RZ(params[j, 1], wires=j)
        return [qml.expval(qml.PauliZ(w)) for w in range(num_qubits)]

    
    features_np = graph.x.numpy()
    encoded_params = quantum_encode(features_np, **flags)
    result = quantum_encoding_circuit(encoded_params)

    if i < 10:  
        print(f"Quantum circuit output for graph {i+1}: {result}")
        print(qml.draw(quantum_encoding_circuit)(encoded_params))


In [ ]:
print(len(dataset_smiles))
print(len(dataset))

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import torch

def quantum_encode(features, use_atomic_num=True, use_nh=True, use_aromaticity=True, use_hybridization=True):
    encoded_params = []
    for feature in features:
        z, nh, aromatic, hybridization = feature.tolist()
        ry_angle, rz_angle = 0, 0

        if use_atomic_num and not use_nh and use_aromaticity and use_hybridization:
            ry_angle = (2 * z - 7) * np.pi / 4 if z is not None else 0
            rz_angle = (-1)**(2 * hybridization - 1) * np.pi / 6 if hybridization is not None else 0
        elif use_atomic_num and use_nh:
            ry_angle = (2 * z - 7) * np.pi / 4 if z is not None else 0
            rz_angle = 2 * np.pi * nh / 5 if nh is not None else 0
        elif use_atomic_num:
            if z in [5, 6, 7]:
                ry_angle = np.cos(-1 / 3)
            if z == 6:
                rz_angle = 2 * np.pi / 3
            elif z == 7:
                rz_angle = -2 * np.pi / 3

        encoded_params.append([ry_angle, rz_angle])
    return np.array(encoded_params)

r0 = np.array(0.1, requires_grad=True)
r1 = np.array(0.2, requires_grad=True)

def setup_circuit(graph, layers=1):
    num_qubits = graph.num_nodes
    dev = qml.device('default.qubit', wires=num_qubits)

    @qml.qnode(dev)
    def quantum_circuit(params, edge_features, edge_indices):
        for i in range(len(params)):
            qml.RY(params[i, 0], wires=i)
            qml.RZ(params[i, 1], wires=i)

        for layer in range(layers):
            if edge_indices.shape[1] > 0:
                for k in range(edge_indices.shape[1]):
                    start, end = edge_indices[0, k], edge_indices[1, k]
                    bond_type = edge_features[k]
                    if bond_type[0] == 1:
                        qml.CNOT(wires=[start, end])
                        qml.RY(params[start][0], wires=end)
                    elif bond_type[1] == 1:
                        qml.CNOT(wires=[start, end])
                        qml.RZ(params[start][1], wires=end)
                    if len(params[start]) > 2 and bond_type[2] == 1:
                        qml.CNOT(wires=[start, end])
                        qml.RX(params[start][2] * layer, wires=end)
                    if len(params[start]) > 3 and bond_type[3] == 1:
                        qml.CNOT(wires=[start, end])
                        qml.Rot(params[start][3] * layer, params[start][3] * layer, params[start][3] * layer, wires=end)

        expectations = [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]
        return expectations

    return quantum_circuit

# number of layers
layers = 3

for i, graph in enumerate(graph_data[:10]):
    quantum_circuit = setup_circuit(graph, layers)
    features_np = graph.x.numpy()
    encoded_params = quantum_encode(features_np)
    edge_features_np = graph.edge_attr.numpy()
    edge_indices_np = graph.edge_index.numpy().T

    if edge_indices_np.size == 0:
        expectations = [0] * num_qubits
    else:
        expectations = quantum_circuit(encoded_params, edge_features_np, edge_indices_np)
    
    mean_expectation = np.mean(expectations)
    readout_result = r0 + r1 * mean_expectation

    print(f"Quantum circuit output for graph {i+1}: {readout_result}")


In [ ]:
import pennylane as qml
from pennylane import numpy as np

def run_optimization(num_qubits, P):
    # Specific edges from your two triangles graph
    edges = [(0, 1), (1, 2), (2, 0),  # First triangle (1-2-3)
             (3, 4), (4, 5), (5, 3),  # Second triangle (4-5-6)
             (2, 6), (3, 6)]          # Connecting edges (3-7 and 4-7)
    
    # Setup the quantum device
    dev = qml.device('default.qubit', wires=num_qubits)

    # Define the Hamiltonians based on the graph structure
    h1 = qml.Hamiltonian([1.0 for _ in edges], [qml.PauliZ(i) @ qml.PauliZ(j) for (i, j) in edges])
    h2 = qml.Hamiltonian([1.0 for _ in range(num_qubits)], [qml.PauliX(i) for i in range(num_qubits)])

    # Define the stabilizers
    stab1 = qml.Hamiltonian([1], [qml.operation.Tensor(*(qml.PauliX(i) for i in range(num_qubits)))])
    stab2 = qml.Hamiltonian([1 for _ in range(num_qubits - 1)], [qml.PauliZ(i) @ qml.PauliZ(i + 1) for i in range(num_qubits - 1)])
    stab = stab1 + stab2

    @qml.qnode(dev)
    def circuit(params):
        for p in range(P):
            qml.templates.ApproxTimeEvolution(h1, params[2 * p], 1)
            qml.templates.ApproxTimeEvolution(h2, params[2 * p + 1], 1)
        return qml.expval(stab)

    params = np.random.random(2 * P)  # 2 parameters per layer

    def loss(params):
        return -circuit(params)

    opt = qml.AdamOptimizer(stepsize=0.01)
    for i in range(500):
        params, val = opt.step_and_cost(loss, params)
        if i % 10 == 0:
            final_expval = circuit(params)  # Evaluate the stabilizer expectation value
            print(f"Step {i}: Stabilizer Expectation = {final_expval}, Loss = {val} for {num_qubits} qubits with {P} layers")

# Set n = 7 and run optimizations for P from 1 to 18
num_qubits = 7
for p in range(1, 19):  # P values from 1 to 18
    print(f"Running optimization for {num_qubits} qubits and {p} layers")
    run_optimization(num_qubits, p)


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create a new graph
G = nx.Graph()

# Add nodes and edges to form two connected triangles
edges = [(1, 2), (2, 3), (3, 1),  # First triangle
         (4, 5), (5, 6), (6, 4),  # Second triangle
         (3, 7), (4,7)                # Connecting edge between the two triangles
         ]                  # Extending second triangle with node 7

G.add_edges_from(edges)

# Position the nodes to reflect the structure with two triangles and an extra node
pos = {1: (0, 1), 2: (1, 2), 3: (2, 1),  # First triangle positioned left
       4: (3, 1), 5: (4, 2), 6: (5, 1),  # Second triangle positioned right
       7: (6, 0)}                        # Node 7 extending from node 6

# Draw the graph
nx.draw(G, pos, with_labels=True, node_color='blue', edge_color='black', node_size=500, font_weight='bold', font_color='white')
plt.title('Graph with Two Connected Triangles and an Extension')
plt.show()


Angle Extraction Network

In [ ]:
import torch
import torch.nn as nn

class AngleExtractionNetwork(nn.Module):
    def __init__(self, input_features, output_features):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_features, 128),
            nn.ReLU(),
            nn.Linear(128, output_features)
        )
    
    def forward(self, x):
        return self.layer(x)


In [ ]:
import pennylane as qml

def setup_quantum_circuit(num_qubits):
    dev = qml.device('default.qubit', wires=num_qubits)

    @qml.qnode(dev)
    def quantum_circuit(angle_params,edge_features, edge_indices):
        for i in range(num_qubits):
            qml.RY(angle_params[i, 0], wires=i)
            qml.RZ(angle_params[i, 1], wires=i)
            
        
        for layer in range(layers):
            if edge_indices.shape[1] > 0:
                for k in range(edge_indices.shape[1]):
                    start, end = edge_indices[0, k], edge_indices[1, k]
                    bond_type = edge_features[k]
                    if bond_type[0] == 1:
                        qml.CNOT(wires=[start, end])
                        qml.RY(angle_params[start][0], wires=end)
                    elif bond_type[1] == 1:
                        qml.CNOT(wires=[start, end])
                        qml.RZ(angle_params[start][1], wires=end)
                    if len(angle_params[start]) > 2 and bond_type[2] == 1:
                        qml.CNOT(wires=[start, end])
                        qml.RX(angle_params[start][2] * layer, wires=end)
                    if len(angle_params[start]) > 3 and bond_type[3] == 1:
                        qml.CNOT(wires=[start, end])
                        qml.Rot(angle_params[start][3] * layer, angle_params[start][3] * layer, angle_params[start][3] * layer, wires=end)

        return [qml.expval(qml.PauliZ(wires=i)) for i in range(num_qubits)]
    

    return quantum_circuit


In [ ]:
def integrate_classical_quantum(graph_data, angle_extractor):
    for i, graph in enumerate(graph_data[:10]):  
        features_np = graph.x.numpy()  
        features_torch = torch.from_numpy(features_np).float()
        
        # Get angles from the neural network
        angles = angle_extractor(features_torch)
        
        
        angles = angles.detach().numpy()
        
        
        quantum_circuit = setup_quantum_circuit(graph.num_nodes)
        edge_features_np = graph.edge_attr.numpy()
        edge_indices_np = graph.edge_index.numpy().T
        if edge_indices_np.size == 0:
            expectations = [0] * num_qubits
        else:
            expectations = quantum_circuit(encoded_params, edge_features_np, edge_indices_np)
    
        
        
        print(f"Quantum circuit output for graph {i+1}: {expectations}")


angle_extractor = AngleExtractionNetwork(input_features=4, output_features=2)
integrate_classical_quantum(graph_data, angle_extractor)


In [ ]:
print(len(graph_data))

In [ ]:
from torch.utils.data import random_split, DataLoader

# Define sizes for training, validation, and testing
train_size = 10000
val_size = 1000
test_size = 1000
total_used_data = train_size + val_size + test_size

# Randomly split the dataset into training, validation, and testing
train_data, remaining_data = random_split(graph_data[:total_used_data], [train_size, val_size + test_size])
val_data, test_data = random_split(remaining_data, [val_size, test_size])

# Loaders for batching the data
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
val_loader = DataLoader(val_data, batch_size=128, shuffle=False)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)




In [ ]:
print(len(dataset_smiles))

In [ ]:
print(len(dataset))

In [ ]:
from torch_geometric.loader import DataLoader
print(DataLoader)


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pennylane as qml

# Define the quantum encoding of features
def quantum_encode(features):
    encoded_params = []
    for graph in features:
        graph_params = []
        for node_features in graph:
            z, nh, aromatic, hybridization = node_features
            if use_atomic_num and not use_nh and use_aromaticity and use_hybridization:
            ry_angle = (2 * z - 7) * np.pi / 4 if z is not None else 0
            rz_angle = (-1)**(2 * hybridization - 1) * np.pi / 6 if hybridization is not None else 0
            elif use_atomic_num and use_nh:
            ry_angle = (2 * z - 7) * np.pi / 4 if z is not None else 0
            rz_angle = 2 * np.pi * nh / 5 if nh is not None else 0
            elif use_atomic_num:
            if z in [5, 6, 7]:
                ry_angle = np.cos(-1 / 3)
            if z == 6:
                rz_angle = 2 * np.pi / 3
            elif z == 7:
                rz_angle = -2 * np.pi / 3

            ry_angle = (2 * z - 7) * np.pi / 4 if z is not None else 0
            rz_angle = (-1)**(2 * hybridization - 1) * np.pi / 6 if hybridization is not None else 0
            graph_params.append([ry_angle, rz_angle])
        encoded_params.append(graph_params)
    return np.array(encoded_params)

# def quantum_encode(features, use_atomic_num=True, use_nh=True, use_aromaticity=True, use_hybridization=True):
#     encoded_params = []
#     for graph in features:
#         graph_params = []
#         for node_features in graph:
#             z, nh, aromatic, hybridization = node_features

#             # Default angles
#             ry_angle, rz_angle = 0, 0

            
#             if use_atomic_num:
#                 ry_angle = (2 * z - 7) * np.pi / 4
#             if use_hybridization:
#                 rz_angle = (-1)**(2 * hybridization - 1) * np.pi / 6
#             if use_nh:
#                 ry_angle += 2 * np.pi * nh / 5  

            
#             graph_params.append([ry_angle, rz_angle])

#         encoded_params.append(graph_params)
#     return np.array(encoded_params)


# # Define your quantum circuit setup
# def setup_circuit(num_qubits, layers=1):
#     dev = qml.device('default.qubit', wires=num_qubits)
#     @qml.qnode(dev)
#     def quantum_circuit(params):
#         # Initial state preparation and parameterized gates
#         for i in range(num_qubits):
#             qml.RY(params[i, 0], wires=i)
#             qml.RZ(params[i, 1], wires=i)
#         # Assume all-to-all connectivity for simplicity, adjust as necessary
#         for i in range(num_qubits):
#             for j in range(i + 1, num_qubits):
#                 qml.CZ(wires=[i, j])
#         return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]
#     return quantum_circuit

import pennylane as qml

import pennylane as qml

# def setup_circuit(num_qubits, edge_features=None, edge_indices=None, layers=1):
#     dev = qml.device('default.qubit', wires=num_qubits)

#     @qml.qnode(dev)
#     def quantum_circuit(params):
#         for i in range(num_qubits):
#             qml.RY(params[i, 0], wires=i)
#             qml.RZ(params[i, 1], wires=i)

#         for layer in range(layers):
#             if graph.edge_index.shape[1] > 0 and graph.edge_attr is not None:
#                 edge_indices = graph.edge_index
#                 edge_features = graph.edge_attr
#                 for k in range(edge_indices.shape[1]):
#                     start, end = edge_indices[0, k], edge_indices[1, k]
#                     bond_type = edge_features[k]
#                     if len(bond_type) > 0 and bond_type[0] == 1:
#                         qml.CNOT(wires=[start, end])
#                         qml.RY(params[start][0], wires=end)
#                     if len(bond_type) > 1 and bond_type[1] == 1:
#                         qml.CNOT(wires=[start, end])
#                         qml.RZ(params[start][1], wires=end)
#                     if len(bond_type) > 2 and bond_type[2] == 1:
#                         qml.CNOT(wires=[start, end])
#                         qml.RX(params[start][2] * layer, wires=end)
#                     if len(bond_type) > 3 and bond_type[3] == 1:
#                         qml.CNOT(wires=[start, end])
#                         qml.Rot(params[start][3] * layer, params[start][3] * layer, params[start][3] * layer, wires=end)

#         return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

#     return quantum_circuit


r0 = torch.tensor(0.1, requires_grad=True)
r1 = torch.tensor(0.2, requires_grad=True)
params = [r0, r1]


# DataLoader setup
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)

# Optimizer
optimizer = torch.optim.Adam(params, lr=0.01)

# Loss function
loss_func = nn.MSELoss()

# # Training loop
# for epoch in range(150):
#     total_loss = 0
#     for features, targets in train_loader:
#         optimizer = optim.Adam([r0, r1], lr=0.01, betas=(0.9, 0.999))
#         optimizer.zero_grad()
#         # Assuming features are properly preprocessed and include graph details
#         num_qubits = features.shape[1]  
        
#         edge_features = np.random.rand(10, 1) 
#         edge_indices = np.random.randint(0, num_qubits, size=(2, 10))  # Example edge indices
#         circuit = setup_circuit(num_qubits, edge_features, edge_indices)
#         expectations = circuit(features.numpy())
#         mean_expectation = np.mean(expectations)
#         readout = r0 + r1 * mean_expectation
#         loss = loss_func(readout, targets)
#         loss.backward()
#         optimizer.step()



for epoch in range(150):
    total_loss = 0
    for graph in train_loader:  
        optimizer.zero_grad()

        if hasattr(graph, 'num_nodes'):
            num_qubits = graph.num_nodes  
            edge_features = graph.edge_attr if hasattr(graph, 'edge_attr') else None
            edge_indices = graph.edge_index if hasattr(graph, 'edge_index') else None

            # Setup and run the quantum circuit
            circuit = setup_circuit(num_qubits, edge_features, edge_indices)
            encoded_features = quantum_encode(graph.x.numpy())  
            expectations = circuit(encoded_features)
            mean_expectation = np.mean(expectations)
            readout = r0 + r1 * mean_expectation
            loss = loss_func(readout, graph.y)  
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        else:
            print("Graph object does not have the required attributes.")

    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Loss {total_loss / len(train_loader)}")





In [ ]:
# DataLoader setup
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)

# Example code to print the first graph in the DataLoader and its attributes
for graph in train_loader:
    print("Graph Attributes:")
    print(dir(graph))  # This will list all attributes and methods of the graph object
    print("Number of Nodes:", graph.num_nodes if hasattr(graph, 'num_nodes') else "Attribute not found")
    print("Edge Index:", graph.edge_index if hasattr(graph, 'edge_index') else "Attribute not found")
    print("Edge Attributes:", graph.edge_attr if hasattr(graph, 'edge_attr') else "Attribute not found")
    print("Node Features (x):", graph.x if hasattr(graph, 'x') else "Attribute not found")
    print("Target (y):", graph.y if hasattr(graph, 'y') else "Attribute not found")
    break


In [ ]:
print(train_data[0]) 